In [ ]:
# %pip install python-dotenv grandalf


Note: you may need to restart the kernel to use updated packages.


In [22]:
# %# 필요한 langchain 패키지들 설치
# %pip install langchain langchain-core langchain-experimental langchain-community langchain-openai langchain-teddynote langchain-huggingface langchain-google-genai langchain-anthropic langchain-cohere langchain-chroma langchain-elasticsearch langchain-upstage langchain-milvus langchain-text-splitters python-dotenv

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from os import environ
from dotenv import load_dotenv

from langchain_core.load import load, loads
from langchain_teddynote import logging

# API KEY 정보로드
load_dotenv()
logging.langsmith("week4-deep")

# import os

# print(f"[API KEY]\n{os.environ['OPENAI_API_KEY'][:-30]}" + "*" * 30)

LangSmith 추적을 시작합니다.
[프로젝트명]
week4-deep


In [24]:
from typing import Dict, List, Any, TypedDict, Literal, Optional, Union
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
import operator
import json

# 상태 타입 정의
class KoreanExamState(TypedDict):
    problem_type: Optional[str]
    context: str
    question: str
    options: str
    context_analysis: Optional[str]
    question_analysis: Optional[str]
    option_evaluations: Optional[List[Dict[str, str]]]
    answer: Optional[str]
    explanation: Optional[str]
    review_notes: Optional[str]
    needs_revision: Optional[bool]
    revision_target: Optional[str]
    revised_analysis: Optional[Dict[str, str]]
    final_response: Optional[str]
    structured_answer: Optional[str]  # 추가: 구조화된 정답만 포함

# GPT-4 모델 초기화
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

# 1. 문제 유형 분류
def classify_problem(state: KoreanExamState) -> KoreanExamState:
    prompt = PromptTemplate.from_template(
        """다음 국어 수능 문제가 어떤 유형인지 분류해주세요:
        1. 문학 (소설, 시, 희곡, 수필 등)
        2. 비문학 (인문, 사회, 과학, 기술 등)
        3. 문법
        4. 화법과 작문
        
        지문: {context}
        문제: {question}
        선택지: {options}
        
        답변 형식: "유형: [번호]"
        """
    )
    
    result = llm.invoke(prompt.format(
        context=state["context"], 
        question=state["question"], 
        options=state["options"]
    ))
    
    problem_type = result.content.strip().split(":")[1].strip()
    return {**state, "problem_type": problem_type}

# 2. 지문 분석
def analyze_context(state: KoreanExamState) -> KoreanExamState:
    # 문제 유형에 따른 프롬프트 선택
    if state["problem_type"] == "1":  # 문학
        prompt_template = """다음 문학 작품 지문을 분석해주세요:
        1. 장르 (소설, 시, 희곡, 수필 등)
        2. 주제
        3. 서술 방식/문체
        4. 주요 인물 또는 화자의 특성
        5. 주요 상징이나 이미지
        6. 작품의 분위기나 정서
        
        지문: {context}
        
        상세하게 분석해주세요.
        """
    elif state["problem_type"] == "2":  # 비문학
        prompt_template = """다음 비문학 지문을 분석해주세요:
        1. 글의 종류 (인문, 사회, 과학, 기술 등)
        2. 주제
        3. 논지 전개 방식
        4. 주요 논점들
        5. 필자의 관점
        6. 핵심 개념이나 이론
        
        지문: {context}
        
        단락별 요약과 함께 상세하게 분석해주세요.
        """
    elif state["problem_type"] == "3":  # 문법
        # 문법 문제는 지문 분석이 필요 없는 경우가 많음
        return {**state, "context_analysis": "문법 문제는 지문 분석이 필요 없습니다."}
    elif state["problem_type"] == "4":  # 화법과 작문
        prompt_template = """다음 화법/작문 관련 지문을 분석해주세요:
        1. 상황 맥락 (대화, 연설, 글쓰기 등)
        2. 화자/필자의 의도
        3. 사용된 주요 화법/작문 전략
        4. 중심 내용
        
        지문: {context}
        
        의사소통 관점에서 상세하게 분석해주세요.
        """
    else:
        prompt_template = """다음 지문을 분석해주세요:
        1. 글의 종류
        2. 주제
        3. 주요 내용
        4. 서술 방식
        
        지문: {context}
        
        상세하게 분석해주세요.
        """
    
    prompt = PromptTemplate.from_template(prompt_template)
    
    # 지문이 없는 경우 처리
    if not state["context"] or state["context"].strip() == "":
        return {**state, "context_analysis": "지문이 제공되지 않았습니다."}
    
    result = llm.invoke(prompt.format(context=state["context"]))
    return {**state, "context_analysis": result.content}

# 3. 문제 분석
def analyze_question(state: KoreanExamState) -> KoreanExamState:
    prompt = PromptTemplate.from_template(
        """다음 국어 수능 문제를 분석해주세요:
        1. 문제가 요구하는 것이 무엇인지
        2. 문제 해결에 필요한 핵심 개념이나 접근 방법
        3. 문제의 난이도
        
        문제: {question}
        
        상세하게 분석해주세요.
        """
    )
    
    result = llm.invoke(prompt.format(question=state["question"]))
    return {**state, "question_analysis": result.content}

# 4. 선택지 평가
def evaluate_options(state: KoreanExamState) -> KoreanExamState:
    prompt_template = """
    다음 국어 수능 문제의 각 선택지를 평가해주세요.
    
    지문: {context}
    문제: {question}
    선택지: {options}
    
    지문 분석: {context_analysis}
    문제 분석: {question_analysis}
    
    각 선택지를 개별적으로 평가하고, 각각이 정답인지 아닌지 판단해주세요.
    평가 결과를 다음 JSON 형식으로 제공해주세요:
    ```json
    [
        {{"option": "①...", "evaluation": "...", "is_correct": true/false}},
        {{"option": "②...", "evaluation": "...", "is_correct": true/false}},
        ...
    ]
    ```
    
    각 선택지에 대해 지문과 문제를 기반으로 철저하게 평가해주세요.
    """
    
    prompt = PromptTemplate.from_template(prompt_template)
    
    result = llm.invoke(prompt.format(
        context=state["context"],
        question=state["question"],
        options=state["options"],
        context_analysis=state["context_analysis"],
        question_analysis=state["question_analysis"]
    ))
    
    # JSON 추출
    json_str = result.content
    if "```json" in json_str:
        json_str = json_str.split("```json")[1].split("```")[0]
    elif "```" in json_str:
        json_str = json_str.split("```")[1].split("```")[0]
    
    try:
        option_evaluations = json.loads(json_str)
    except:
        # JSON 파싱 실패 시 텍스트 그대로 반환
        return {**state, "option_evaluations": [{"error": "JSON 파싱 실패", "raw": result.content}]}
    
    return {**state, "option_evaluations": option_evaluations}

# 5. 답안 도출
def determine_answer(state: KoreanExamState) -> KoreanExamState:
    # 옵션 평가에서 정답 찾기
    answer = None
    for option in state["option_evaluations"]:
        if "is_correct" in option and option["is_correct"]:
            # 선택지에서 번호 추출 (①, ②, ③, ④, ⑤)
            answer = option["option"][0]  # 첫 글자(번호) 추출
            break
    
    if not answer:
        # 정답을 명확히 찾지 못한 경우, 다시 분석
        prompt = PromptTemplate.from_template(
            """다음 국어 수능 문제와 분석 결과를 바탕으로 최종 답안을 선택해주세요:
            
            지문: {context}
            문제: {question}
            선택지: {options}
            
            지문 분석: {context_analysis}
            문제 분석: {question_analysis}
            선택지 평가: {option_evaluations}
            
            최종 답안(①, ②, ③, ④, ⑤ 중 하나)만 제시해주세요.
            """
        )
        
        result = llm.invoke(prompt.format(
            context=state["context"],
            question=state["question"],
            options=state["options"],
            context_analysis=state["context_analysis"],
            question_analysis=state["question_analysis"],
            option_evaluations=str(state["option_evaluations"])
        ))
        
        answer = result.content.strip()
    
    return {**state, "answer": answer}

# 6. 답안 설명
def explain_answer(state: KoreanExamState) -> KoreanExamState:
    prompt = PromptTemplate.from_template(
        """다음 국어 수능 문제에 대한 최종 답안을 설명해주세요:
        
        지문: {context}
        문제: {question}
        선택지: {options}
        
        선택한 답안: {answer}
        
        다음을 포함하여 설명해주세요:
        1. 왜 그 답안이 정답인지에 대한 상세한 설명
        2. 다른 선택지들이 왜 오답인지에 대한 설명
        3. 지문에서 정답을 뒷받침하는 구체적인 근거
        
        명확하고 논리적으로 설명해주세요.
        """
    )
    
    result = llm.invoke(prompt.format(
        context=state["context"],
        question=state["question"],
        options=state["options"],
        answer=state["answer"]
    ))
    
    return {**state, "explanation": result.content}

# 7. 검토 과정 - 모든 분석과 추론을 검토
def review_analysis(state: KoreanExamState) -> KoreanExamState:
    prompt = PromptTemplate.from_template(
        """국어 수능 문제 풀이 과정을 철저히 검토해주세요. 다음 각 분석 단계에서 오류나 개선할 점이 있는지 평가하세요:
        
        지문: {context}
        문제: {question}
        선택지: {options}
        
        지문 분석: {context_analysis}
        문제 분석: {question_analysis}
        선택지 평가: {option_evaluations}
        도출된 답안: {answer}
        답안 설명: {explanation}
        
        다음 항목들을 중점적으로 검토하세요:
        1. 지문에 대한 이해가 정확한가?
        2. 문제의 요구사항을 제대로 파악했는가?
        3. 선택지 평가가 논리적이고 일관성이 있는가?
        4. 답안 선택의 근거가 충분한가?
        5. 지문의 맥락과 문제의 의도를 제대로 반영했는가?
        
        검토 결과를 다음 형식으로 제공해주세요:
        ```json
        {{
            "needs_revision": true/false,
            "revision_target": "context_analysis/question_analysis/option_evaluations/answer/explanation",
            "review_notes": "검토 내용과 개선 제안"
        }}
        ```
        
        철저하고 비판적인 시각으로 검토해주세요.
        """
    )
    
    result = llm.invoke(prompt.format(
        context=state["context"],
        question=state["question"],
        options=state["options"],
        context_analysis=state["context_analysis"],
        question_analysis=state["question_analysis"],
        option_evaluations=str(state["option_evaluations"]),
        answer=state["answer"],
        explanation=state["explanation"]
    ))
    
    # JSON 추출
    json_str = result.content
    if "```json" in json_str:
        json_str = json_str.split("```json")[1].split("```")[0]
    elif "```" in json_str:
        json_str = json_str.split("```")[1].split("```")[0]
    
    try:
        review_result = json.loads(json_str)
        return {**state, 
                "review_notes": review_result.get("review_notes", "검토 완료"),
                "needs_revision": review_result.get("needs_revision", False),
                "revision_target": review_result.get("revision_target", None)}
    except:
        # JSON 파싱 실패 시 기본값 설정
        return {**state, 
                "review_notes": "검토 결과 파싱 실패: " + result.content,
                "needs_revision": False,
                "revision_target": None}

# 8. 분석 수정 - 검토 결과에 따른 수정 작업
def revise_analysis(state: KoreanExamState) -> KoreanExamState:
    target = state["revision_target"]
    review_notes = state["review_notes"]
    
    if target == "context_analysis":
        prompt_template = """
        국어 수능 지문 분석을 다시 수행해주세요. 이전 분석에서 다음과 같은 문제점이 발견되었습니다:
        
        문제점: {review_notes}
        
        지문: {context}
        
        이전 분석: {context_analysis}
        
        위 문제점을 해결하여 더 정확하고 심층적인 지문 분석을 제공해주세요.
        """
        
        prompt = PromptTemplate.from_template(prompt_template)
        result = llm.invoke(prompt.format(
            review_notes=review_notes,
            context=state["context"],
            context_analysis=state["context_analysis"]
        ))
        
        return {**state, "revised_analysis": {"context_analysis": result.content}}
        
    elif target == "question_analysis":
        prompt_template = """
        국어 수능 문제 분석을 다시 수행해주세요. 이전 분석에서 다음과 같은 문제점이 발견되었습니다:
        
        문제점: {review_notes}
        
        문제: {question}
        
        이전 분석: {question_analysis}
        
        위 문제점을 해결하여 더 정확하고 심층적인 문제 분석을 제공해주세요.
        """
        
        prompt = PromptTemplate.from_template(prompt_template)
        result = llm.invoke(prompt.format(
            review_notes=review_notes,
            question=state["question"],
            question_analysis=state["question_analysis"]
        ))
        
        return {**state, "revised_analysis": {"question_analysis": result.content}}
        
    elif target == "option_evaluations":
        prompt_template = """
        국어 수능 선택지 평가를 다시 수행해주세요. 이전 평가에서 다음과 같은 문제점이 발견되었습니다:
        
        문제점: {review_notes}
        
        지문: {context}
        문제: {question}
        선택지: {options}
        
        이전 평가: {option_evaluations}
        
        위 문제점을 해결하여 더 정확하고 논리적인 선택지 평가를 제공해주세요.
        평가 결과를 다음 JSON 형식으로 제공해주세요:
        ```json
        [
            {{"option": "①...", "evaluation": "...", "is_correct": true/false}},
            {{"option": "②...", "evaluation": "...", "is_correct": true/false}},
            ...
        ]
        ```
        """
        
        prompt = PromptTemplate.from_template(prompt_template)
        result = llm.invoke(prompt.format(
            review_notes=review_notes,
            context=state["context"],
            question=state["question"],
            options=state["options"],
            option_evaluations=str(state["option_evaluations"])
        ))
        
        # JSON 추출
        json_str = result.content
        if "```json" in json_str:
            json_str = json_str.split("```json")[1].split("```")[0]
        elif "```" in json_str:
            json_str = json_str.split("```")[1].split("```")[0]
        
        try:
            revised_evaluations = json.loads(json_str)
            return {**state, "revised_analysis": {"option_evaluations": revised_evaluations}}
        except:
            return {**state, "revised_analysis": {"option_evaluations": "JSON 파싱 실패: " + result.content}}
        
    elif target == "answer":
        prompt_template = """
        국어 수능 문제의 답안을 다시 검토해주세요. 이전 답안 선택에서 다음과 같은 문제점이 발견되었습니다:
        
        문제점: {review_notes}
        
        지문: {context}
        문제: {question}
        선택지: {options}
        
        지문 분석: {context_analysis}
        문제 분석: {question_analysis}
        선택지 평가: {option_evaluations}
        
        이전 답안: {answer}
        
        위 문제점을 해결하여 더 정확한 답안을 선택해주세요.
        """
        
        prompt = PromptTemplate.from_template(prompt_template)
        result = llm.invoke(prompt.format(
            review_notes=review_notes,
            context=state["context"],
            question=state["question"],
            options=state["options"],
            context_analysis=state["context_analysis"],
            question_analysis=state["question_analysis"],
            option_evaluations=str(state["option_evaluations"]),
            answer=state["answer"]
        ))
        
        return {**state, "revised_analysis": {"answer": result.content.strip()}}
        
    elif target == "explanation":
        prompt_template = """
        국어 수능 문제의 답안 설명을 다시 작성해주세요. 이전 설명에서 다음과 같은 문제점이 발견되었습니다:
        
        문제점: {review_notes}
        
        지문: {context}
        문제: {question}
        선택지: {options}
        답안: {answer}
        
        이전 설명: {explanation}
        
        위 문제점을 해결하여 더 명확하고 논리적인 답안 설명을 제공해주세요.
        """
        
        prompt = PromptTemplate.from_template(prompt_template)
        result = llm.invoke(prompt.format(
            review_notes=review_notes,
            context=state["context"],
            question=state["question"],
            options=state["options"],
            answer=state["answer"],
            explanation=state["explanation"]
        ))
        
        return {**state, "revised_analysis": {"explanation": result.content}}
    
    else:
        # 수정 대상이 명확하지 않은 경우
        return {**state, "revised_analysis": {"note": "수정 대상이 명확하지 않습니다."}}

# 9. 수정사항 적용
def apply_revisions(state: KoreanExamState) -> KoreanExamState:
    revised = state.get("revised_analysis", {})
    
    updated_state = {**state}
    
    # 각 수정사항을 적용
    if "context_analysis" in revised:
        updated_state["context_analysis"] = revised["context_analysis"]
    
    if "question_analysis" in revised:
        updated_state["question_analysis"] = revised["question_analysis"]
    
    if "option_evaluations" in revised:
        if isinstance(revised["option_evaluations"], list):
            updated_state["option_evaluations"] = revised["option_evaluations"]
        else:
            # JSON 파싱 실패한 경우 원본 유지
            pass
    
    if "answer" in revised:
        updated_state["answer"] = revised["answer"]
    
    if "explanation" in revised:
        updated_state["explanation"] = revised["explanation"]
    
    # 수정 후 재검토 필요 여부 초기화
    updated_state["needs_revision"] = False
    updated_state["revision_target"] = None
    
    return updated_state

# 10. 최종 응답 생성 - 상세한 최종 응답과 구조화된 정답을 모두 생성
def generate_final_response(state: KoreanExamState) -> KoreanExamState:
    # 기존의 상세한 응답 생성
    option_evaluations_str = ""
    for option in state.get("option_evaluations", []):
        option_evaluations_str += f"- {option['option']}: {option['evaluation']}\n"
    
    final_response = f"""
    # 국어 수능 문제 분석 결과
    
    ## 문제 유형
    {state["problem_type"]}
    
    ## 문제 분석
    {state["question_analysis"]}
    
    ## 지문 분석
    {state["context_analysis"]}
    
    ## 선택지 평가
    {option_evaluations_str}
    
    ## 검토 내용
    {state.get("review_notes", "검토 완료")}
    
    ## 최종 답안
    {state["answer"]}
    
    ## 답안 설명
    {state["explanation"]}
    """
    
    # 구조화된 정답 생성 (①, ②, ③, ④, ⑤를 1, 2, 3, 4, 5로 변환)
    answer = state["answer"]
    structured_answer = None
    
    # 원본 답안(①, ②, ③, ④, ⑤)을 숫자(1, 2, 3, 4, 5)로 변환
    if "①" in answer:
        structured_answer = "1"
    elif "②" in answer:
        structured_answer = "2"
    elif "③" in answer:
        structured_answer = "3"
    elif "④" in answer:
        structured_answer = "4"
    elif "⑤" in answer:
        structured_answer = "5"
    else:
        # 답안에서 숫자 추출 시도
        import re
        number_match = re.search(r'(\d+)', answer)
        if number_match:
            structured_answer = number_match.group(1)
        else:
            structured_answer = "답안 형식 오류"
    
    return {**state, "final_response": final_response, "structured_answer": structured_answer}

# 라우터 함수 수정 - 다음 단계를 "next" 키에 저장
def router(state: KoreanExamState) -> KoreanExamState:
    next_step = None
    
    # 검토 과정 처리
    if state.get("needs_revision") == True and state.get("revision_target") and not state.get("revised_analysis"):
        next_step = "revise_analysis"
    elif state.get("revised_analysis") and state.get("needs_revision") == True:
        next_step = "apply_revisions"
    
    # 문법 문제는 지문 분석 단계를 건너뛸 수 있음
    elif state["problem_type"] == "3" and not state.get("question_analysis"):
        next_step = "analyze_question"
    
    # 기본 플로우
    elif not state.get("context_analysis"):
        next_step = "analyze_context"
    elif not state.get("question_analysis"):
        next_step = "analyze_question"
    elif not state.get("option_evaluations"):
        next_step = "evaluate_options"
    elif not state.get("answer"):
        next_step = "determine_answer"
    elif not state.get("explanation"):
        next_step = "explain_answer"
    elif not state.get("review_notes"):
        next_step = "review_analysis"
    elif not state.get("final_response"):
        next_step = "generate_final_response"
    else:
        next_step = END
    
    return {**state, "next": next_step}

# 그래프 구성 - 조건부 엣지 수정
workflow = StateGraph(KoreanExamState)

# 노드 추가
workflow.add_node("classify_problem", classify_problem)
workflow.add_node("analyze_context", analyze_context)
workflow.add_node("analyze_question", analyze_question)
workflow.add_node("evaluate_options", evaluate_options)
workflow.add_node("determine_answer", determine_answer)
workflow.add_node("explain_answer", explain_answer)
workflow.add_node("review_analysis", review_analysis)
workflow.add_node("revise_analysis", revise_analysis)
workflow.add_node("apply_revisions", apply_revisions)
workflow.add_node("generate_final_response", generate_final_response)
workflow.add_node("router", router)

# 조건부 엣지 설정 - 수정된 방식
workflow.set_entry_point("classify_problem")
workflow.add_edge("classify_problem", "router")

# 조건부 엣지 대신 조건자 딕셔너리 사용
workflow.add_conditional_edges(
    "router",
    lambda x: x.get("next", ""),
    {
        "analyze_context": "analyze_context",
        "analyze_question": "analyze_question",
        "evaluate_options": "evaluate_options",
        "determine_answer": "determine_answer",
        "explain_answer": "explain_answer",
        "review_analysis": "review_analysis",
        "revise_analysis": "revise_analysis",
        "apply_revisions": "apply_revisions",
        "generate_final_response": "generate_final_response",
        END: END
    }
)

workflow.add_edge("analyze_context", "router")
workflow.add_edge("analyze_question", "router")
workflow.add_edge("evaluate_options", "router")
workflow.add_edge("determine_answer", "router")
workflow.add_edge("explain_answer", "router")
workflow.add_edge("review_analysis", "router")
workflow.add_edge("revise_analysis", "router")
workflow.add_edge("apply_revisions", "router")
workflow.add_edge("generate_final_response", "router")


# 실행 코드 수정 - 구조화된 정답만 반환
def solve_korean_problem(context, question, options):
    app = workflow.compile()
    inputs = {
        "context": context,
        "question": question,
        "options": options
    }
    result = app.invoke(inputs)
    
    # 구조화된 정답만 반환
    return result.get("structured_answer", "분석 실패")

# 기존 추론 과정을 모두 유지하면서 결과도 확인하기 위한 함수 (디버깅용)
def solve_korean_problem_with_details(context, question, options):
    app = workflow.compile()
    inputs = {
        "context": context,
        "question": question,
        "options": options
    }
    result = app.invoke(inputs)
    
    # 전체 결과 반환(디버깅용)
    return result

In [25]:
graph = workflow.compile()
print(graph.get_graph().draw_ascii())


                                                                                                                                        +-----------+                                                                                                                                      
                                                                                                                                        | __start__ |                                                                                                                                      
                                                                                                                                        +-----------+                                                                                                                                      
                                                                                                                                               *    

In [ ]:

    
    result = solve_korean_problem(sample_context, sample_question, sample_options)
    print(result)

2
